# Collecting the first 500 pages of movie titles ordered by most popular 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import csv
import re

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)



fields = ["Link", "Title", "Year", "Rating"] # creating data columns
filename = "Letterboxd_scrape_first500.csv" # new file
# creating lists 
rows = []
links = []
titles_temp = [] #temporary container for movie info 
titles = []
year = []
rating = []
errors = []
synopsis = []

for i in range(1,501):
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 
            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            link = movie.find_element(By.TAG_NAME, "a").get_attribute('href')
            links.append(link) # link

    except: #print page number of where error occurs
        errors.append(i)
        continue
for i in errors:
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 

            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            link = movie.find_element(By.TAG_NAME, "a").get_attribute('href')
            links.append(link) # link

    except: #print page number of where error occurs
        print("page: "+str(i))
        continue

for film in titles_temp: #iterate through movie info for each one
    try:
    # append separate features of the movie info 
        titles.append(film[:-11].strip()) 
        year.append(film[-10:-6].strip())
        rating.append(film[-4:].strip())
    # print movie info when there's an error to track it 
    except IndexError:
        print(new_list)
        print(film)
        continue
for i in range(len(titles)-1): 
    try:
        rows.append([links[i], titles[i], year[i], rating[i]]) # create rows of csv 
    except IndexError:
        print(i) # print index where error occurs 
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows)

driver.quit()
print("Done")

# Collecting synopsis for each movie title
### (This data ended up not being utilized)

In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
import requests_cache
import csv
import re
import pandas as pd
import cchardet as chardet
from multiprocessing import Pool

#for speeding up BeautifulSoup requests by saving cache for 24 hours
requests_cache.install_cache('my_cache', expire_after=86400)
requests_session = requests.Session()

df = pd.read_csv('Letterboxd_scrape_first500.csv')
synopsis = []
for link in df["Link"]: #get each link from dataframe
    try:
        
        source = requests_session.get(link)
        soup = BeautifulSoup(source.content, 'lxml')
        #finds where the synopsis is located and get the text
        main = soup.find("div", class_ = "content-wrap")
        main = main.find("div", class_ = "review body-text -prose -hero prettify")
        synop = main.find("div", class_ = 'truncate').p.text 
        #add to synopsis list
        synopsis.append(synop)
    except:
        #if synopsis is unavailable, add nothing to the list
        synopsis.append("")
        continue
#add the synopsis list into a dataframe column
df['Synopsis'] = pd.Series(synopsis)
df.to_csv('Letterboxd_scrape_first500_v2.csv')  
print("Done")